In [65]:
import psycopg2
import pandas as pd
import geopandas as gpd
from jinja2 import Environment, FileSystemLoader
from config import config

params = config()
conn = psycopg2.connect(**params)

a = !ls ../FINALES/PARCELAS/TORNAVACAS/*shp
parcelas = list(map(lambda x: x.split('/')[-1].split('.')[0], a))
parcelas_dict = []
for parcela in parcelas:
#     for index, value in p.iterrows():
#         display(value.geom)
#     # SUPERFICIE
    
    sup = gpd.read_postgis('''
                SELECT
                    geom,
                    ST_Area(ST_Transform(a.geom, 25830)) / 10000 as sup, 
                    ST_Perimeter(ST_Transform(a.geom, 25830)) as perim,
                    (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).min as pend_min,
                    (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).max as pend_max,
                    (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).mean as pend_mean,
                    (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).min as alt_min,
                    (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).max as alt_max,
                    (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).mean as alt_mean,
                    (ST_SummaryStats(St_Union(ST_Clip(d.rast, st_transform(geom, 25830))))).mean as orient
                FROM 
                    {} a,
                    pendientes25 b,
                    mdt25 c,
                    orientaciones25 d
                GROUP BY
                    a.geom
                '''.format(parcela), conn)
    parcelas_dict.append({'nombre': parcela, 'datos':sup.to_dict(orient='records')})
env = Environment(loader=FileSystemLoader('static/ficha_ejecucion'))
template =env.get_template('base.html')
output = template.render(
    parcelas = parcelas_dict
)

with open('static/ficha_ejecucion/ficha_ejecucion.html', 'w') as f:
    f.write(output)